Census of Drug and Alcohol Treatment Services in Northern Ireland:Breakdown by Service Type

In [1]:
from gssutils import *
if is_interactive():
    import requests
    from cachecontrol import CacheControl
    from cachecontrol.caches.file_cache import FileCache
    from cachecontrol.heuristics import LastModified
    from pathlib import Path

    session = CacheControl(requests.Session(),
                           cache=FileCache('.cache'),
                           heuristic=LastModified())

    sourceFolder = Path('in')
    sourceFolder.mkdir(exist_ok=True)

    inputURL = 'https://www.health-ni.gov.uk/sites/default/files/publications/dhssps/data-census-drug-alcohol-treatment-services.xlsx'
    inputFile = sourceFolder / 'data-census-drug-alcohol-treatment-services.xlsx'
    response = session.get(inputURL)
    with open(inputFile, 'wb') as f:
      f.write(response.content)
    tab = loadxlstabs(inputFile, sheetids='Table 2')[0]

Loading in\data-census-drug-alcohol-treatment-services.xlsx which has size 46265 bytes
Table names: ['Table 2']


In [2]:
observations = tab.excel_ref('B16').expand(DOWN).expand(RIGHT).is_not_blank() - tab.excel_ref('B22').expand(DOWN).expand(RIGHT)  


In [3]:
observations

{<K18 0.0>, <H15 1062.0>, <I15 5256.0>, <L15 1356.0>, <F18 '*'>, <I17 1606.0>, <K20 35.6>, <G18 '*'>, <G15 1712.0>, <C15 324.0>, <D16 '*'>, <G16 '*'>, <C20 90.7>, <H16 '*'>, <J15 2577.0>, <B21 '-'>, <M16 3600.0>, <I18 '*'>, <K17 724.0>, <F19 '*'>, <M19 60.3>, <H20 38.3>, <F17 769.0>, <H19 '*'>, <J21 0.2>, <J19 66.7>, <H18 '*'>, <G17 430.0>, <H17 407.0>, <B18 '-'>, <F21 '*'>, <K21 0.0>, <C17 294.0>, <K19 64.4>, <B20 88.4>, <B19 '*'>, <B17 84.0>, <I16 '*'>, <E18 '-'>, <D20 68.7>, <L17 609.0>, <M21 3.1>, <L19 42.0>, <K15 2036.0>, <D19 '*'>, <M15 5969.0>, <D17 202.0>, <I20 30.6>, <E20 81.3>, <M20 36.6>, <E19 '*'>, <H21 '*'>, <C19 '*'>, <E21 '-'>, <I19 '*'>, <L21 13.1>, <B15 95.0>, <G20 25.1>, <E15 713.0>, <I21 '*'>, <L20 44.9>, <C16 '*'>, <F16 '*'>, <M18 183.0>, <M17 2186.0>, <G21 '*'>, <J16 1719.0>, <D15 294.0>, <E17 580.0>, <G19 '*'>, <J17 853.0>, <L18 178.0>, <L16 569.0>, <K16 1312.0>, <F20 31.0>, <B16 '*'>, <E16 '*'>, <J18 5.0>, <F15 2482.0>, <J20 33.1>}

In [4]:
Service = tab.excel_ref('A15').expand(DOWN).is_not_blank()
Service

{<A18 'Prison'>, <A20 'Non-statutory (%)'>, <A15 'Total'>, <A17 'Non-statutory'>, <A21 'Prison (%)'>, <A16 'Statutory'>, <A19 'Statutory (%)'>}

In [5]:
Treatment = tab.excel_ref('B14').expand(RIGHT).is_not_blank()
Treatment

{<I14 'Total'>, <H14 'Drugs & Alcohol'>, <B14 'Alcohol Only'>, <K14 'Drugs Only'>, <E14 'Total'>, <J14 'Alcohol Only'>, <D14 'Drugs & Alcohol'>, <C14 'Drugs Only'>, <L14 'Drugs & Alcohol'>, <F14 'Alcohol Only'>, <G14 'Drugs Only'>}

In [6]:
age = tab.excel_ref('B13').expand(RIGHT).is_not_blank()
age

{<B13 'Under 18 '>, <M13 'Overall Total'>, <J13 'Treatment Type'>, <F13 '18 and over'>}

In [7]:
Dimensions = [
            HDim(Treatment,'Treatment Type',DIRECTLY,ABOVE),
            HDim(Service,'Service Type',DIRECTLY,LEFT),
            HDim(age,'Age',CLOSEST,LEFT),
            HDimConst('Measure Type', 'Count'),
            HDimConst('Unit','People'),
            HDimConst('Period','1 March 2017'),
            HDimConst('Sex','Persons')
            ]

In [8]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
# savepreviewhtml(c1)

In [9]:
new_table = c1.topandas()
new_table

,OBS,DATAMARKER,Treatment Type,Service Type,Age,Measure Type,Unit,Period,Sex
0,95,NaN,Alcohol Only,Total,Under 18,Count,People,1 March 2017,Persons
1,324,NaN,Drugs Only,Total,Under 18,Count,People,1 March 2017,Persons
2,294,NaN,Drugs & Alcohol,Total,Under 18,Count,People,1 March 2017,Persons
3,713,NaN,Total,Total,Under 18,Count,People,1 March 2017,Persons
4,2482,NaN,Alcohol Only,Total,18 and over,Count,People,1 March 2017,Persons
5,1712,NaN,Drugs Only,Total,18 and over,Count,People,1 March 2017,Persons
6,1062,NaN,Drugs & Alcohol,Total,18 and over,Count,People,1 March 2017,Persons
7,5256,NaN,Total,Total,18 and over,Count,People,1 March 2017,Persons
8,2577,NaN,Alcohol Only,Total,Treatment Type,Count,People,1 March 2017,Persons
9,2036,NaN,Drugs Only,Total,Treatment Type,Count,People,1 March 2017,Persons


In [10]:
new_table.columns = ['Value' if x=='OBS' else x for x in new_table.columns]

In [11]:
new_table.dtypes

Value             object
DATAMARKER        object
Treatment Type    object
Service Type      object
Age               object
Measure Type      object
Unit              object
Period            object
Sex               object
dtype: object

In [12]:
new_table.tail(5)

,Value,DATAMARKER,Treatment Type,Service Type,Age,Measure Type,Unit,Period,Sex
75,,*,Total,Prison (%),18 and over,Count,People,1 March 2017,Persons
76,0.2,NaN,Alcohol Only,Prison (%),Treatment Type,Count,People,1 March 2017,Persons
77,0,NaN,Drugs Only,Prison (%),Treatment Type,Count,People,1 March 2017,Persons
78,13.1,NaN,Drugs & Alcohol,Prison (%),Treatment Type,Count,People,1 March 2017,Persons
79,3.1,NaN,None,Prison (%),Overall Total,Count,People,1 March 2017,Persons


In [13]:
new_table.count()

Value             80
DATAMARKER        28
Treatment Type    73
Service Type      80
Age               80
Measure Type      80
Unit              80
Period            80
Sex               80
dtype: int64

In [14]:
new_table = new_table[new_table['Value'] !=  0 ]

In [15]:
new_table = new_table[new_table['Value'] !=  '' ]

In [16]:
new_table.count()

Value             50
DATAMARKER         0
Treatment Type    43
Service Type      50
Age               50
Measure Type      50
Unit              50
Period            50
Sex               50
dtype: int64

In [17]:
def user_perc(x):
    
    if str(x) == 'Treatment Type':
        return 'All years'
    else:
        return x
    
new_table['Age'] = new_table.apply(lambda row: user_perc(row['Age']), axis = 1)

In [18]:
new_table['Treatment Type'].fillna('All', inplace = True)
# new_table['Service Type'] = 'All'
new_table['Residential Status'] = 'All'
new_table['Health and Social Care Trust']  = 'All'

In [19]:
new_table = new_table[['Period', 'Sex', 'Age', 'Service Type', 'Residential Status', 'Treatment Type', 'Health and Social Care Trust', 'Measure Type', 'Unit', 'Value']]

In [20]:
new_table.head(5)

,Period,Sex,Age,Service Type,Residential Status,Treatment Type,Health and Social Care Trust,Measure Type,Unit,Value
0,1 March 2017,Persons,Under 18,Total,All,Alcohol Only,All,Count,People,95
1,1 March 2017,Persons,Under 18,Total,All,Drugs Only,All,Count,People,324
2,1 March 2017,Persons,Under 18,Total,All,Drugs & Alcohol,All,Count,People,294
3,1 March 2017,Persons,Under 18,Total,All,Total,All,Count,People,713
4,1 March 2017,Persons,18 and over,Total,All,Alcohol Only,All,Count,People,2482
